### Diagnosing breast cancer using KNN Algorithm

##### Load the dataset

In [185]:
wbcd <- read.csv("wisc_bc_data.csv", stringsAsFactors = FALSE)
head(wbcd)

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
87139402,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
8910251,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
905520,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
868871,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
9012568,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766
906539,B,11.57,19.04,74.20,409.7,0.08546,0.07722,0.05485,0.01428,...,13.07,26.98,86.43,520.5,0.1249,0.1937,0.25600,0.06664,0.3035,0.08284


In [186]:
str(wbcd)

'data.frame':	569 obs. of  32 variables:
 $ id               : int  87139402 8910251 905520 868871 9012568 906539 925291 87880 862989 89827 ...
 $ diagnosis        : chr  "B" "B" "B" "B" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  1

In [187]:
# Drop the Id column
wbcd <- wbcd[-1]

In [188]:
head(wbcd)

diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766
B,11.57,19.04,74.20,409.7,0.08546,0.07722,0.05485,0.01428,0.2031,...,13.07,26.98,86.43,520.5,0.1249,0.1937,0.25600,0.06664,0.3035,0.08284


In [189]:
table(wbcd$diagnosis)


  B   M 
357 212 

In [190]:
# target feature diagnosis coded as factor
wbcd$diagnosis <- factor(x = wbcd$diagnosis, levels = c("B", "M"),labels = c("Benign", "Malignant"))

In [191]:
wbcd$diagnosis

[1] Benign    Benign    Benign    Benign    Benign    Benign    Benign   
  [8] Malignant Benign    Benign    Malignant Benign    Benign    Benign   
 [15] Malignant Benign    Benign    Benign    Malignant Benign    Benign   
 [22] Benign    Benign    Benign    Benign    Malignant Benign    Malignant
 [29] Benign    Benign    Benign    Malignant Malignant Benign    Benign   
 [36] Benign    Malignant Benign    Malignant Malignant Malignant Malignant
 [43] Malignant Benign    Benign    Malignant Benign    Malignant Benign   
 [50] Benign    Malignant Benign    Benign    Benign    Malignant Benign   
 [57] Benign    Benign    Malignant Malignant Malignant Malignant Malignant
 [64] Malignant Malignant Benign    Benign    Benign    Benign    Benign   
 [71] Malignant Benign    Benign    Benign    Benign    Benign    Malignant
 [78] Benign    Benign    Malignant Benign    Benign    Benign    Benign   
 [85] Benign    Benign    Benign    Benign    Malignant Benign    Benign   
 [92] Benign    Malignant Benign    Malignant Malignant Benign    Benign   
 [99] Benign    Malignant Benign    Benign    Benign    Malignant Benign   
[106] Benign    Benign    Benign    Benign    Benign    Benign    Benign   
[113] Malignant Malignant Benign    Malignant Malignant Malignant Malignant
[120] Benign    Benign    Malignant Benign    Malignant Benign    Benign   
[127] Malignant Malignant Malignant Malignant Malignant Benign    Benign   
[134] Malignant Benign    Malignant Benign    Benign    Malignant Benign   
[141] Malignant Malignant Malignant Malignant Malignant Benign    Malignant
[148] Benign    Benign    Benign    Benign    Benign    Benign    Benign   
[155] Benign    Benign    Benign    Malignant Benign    Benign    Benign   
[162] Benign    Benign    Benign    Malignant Malignant Benign    Malignant
[169] Benign    Benign    Malignant Malignant Malignant Benign    Malignant
[176] Benign    Malignant Benign    Malignant Benign    Benign    Malignant
[183] Benign    Malignant Benign    Benign    Malignant Malignant Malignant
[190] Benign    Malignant Benign    Benign    Malignant Malignant Malignant
[197] Malignant Benign    Benign    Benign    Malignant Benign    Benign   
[204] Benign    Benign    Malignant Benign    Malignant Benign    Malignant
[211] Benign    Malignant Malignant Benign    Benign    Benign    Benign   
[218] Benign    Malignant Malignant Benign    Benign    Malignant Benign   
[225] Benign    Benign    Benign    Malignant Malignant Benign    Benign   
[232] Benign    Benign    Malignant Benign    Benign    Malignant Benign   
[239] Benign    Benign    Malignant Benign    Benign    Malignant Malignant
[246] Benign    Benign    Benign    Benign    Benign    Benign    Malignant
[253] Benign    Malignant Benign    Benign    Benign    Benign    Benign   
[260] Benign    Benign    Benign    Malignant Benign    Malignant Malignant
[267] Benign    Benign    Benign    Malignant Benign    Malignant Benign   
[274] Benign    Malignant Malignant Benign    Malignant Benign    Benign   
[281] Malignant Benign    Benign    Benign    Malignant Benign    Malignant
[288] Malignant Malignant Benign    Benign    Benign    Malignant Benign   
[295] Malignant Malignant Malignant Malignant Benign    Malignant Malignant
[302] Benign    Benign    Malignant Malignant Malignant Malignant Benign   
[309] Benign    Benign    Malignant Malignant Benign    Benign    Malignant
[316] Benign    Malignant Malignant Benign    Benign    Benign    Malignant
[323] Malignant Benign    Benign    Malignant Benign    Malignant Benign   
[330] Malignant Benign    Benign    Benign    Benign    Malignant Benign   
[337] Benign    Benign    Malignant Benign    Benign    Malignant Malignant
[344] Benign    Malignant Benign    Benign    Benign    Malignant Benign   
[351] Benign    Malignant Benign    Benign    Malignant Benign    Benign   
[358] Benign    Malignant Benign    Malignant Malignant Benign    Benign   
[365] Benign    Benign    Malignant Benign    Beni

In [192]:
head(wbcd)

diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
Benign,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
Benign,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
Benign,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
Benign,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
Benign,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766
Benign,11.57,19.04,74.20,409.7,0.08546,0.07722,0.05485,0.01428,0.2031,...,13.07,26.98,86.43,520.5,0.1249,0.1937,0.25600,0.06664,0.3035,0.08284


In [193]:
round(prop.table(table(wbcd$diagnosis)) * 100, digits=1)


   Benign Malignant 
     62.7      37.3 

In [194]:
summary(wbcd[c("radius_mean", "area_mean", "smoothness_mean")])

  radius_mean       area_mean      smoothness_mean  
 Min.   : 6.981   Min.   : 143.5   Min.   :0.05263  
 1st Qu.:11.700   1st Qu.: 420.3   1st Qu.:0.08637  
 Median :13.370   Median : 551.1   Median :0.09587  
 Mean   :14.127   Mean   : 654.9   Mean   :0.09636  
 3rd Qu.:15.780   3rd Qu.: 782.7   3rd Qu.:0.10530  
 Max.   :28.110   Max.   :2501.0   Max.   :0.16340  

##### Transformation - Normalizing numeric data

In [195]:
# Craete a normalize function
normalize <- function(x) {
    return ((x - min(x)) / (max(x) - min(x)))
}

In [196]:
normalize (c(1,2,3,4,5))

[1] 0.00 0.25 0.50 0.75 1.00

In [197]:
normalize (c(10, 20, 30, 40, 50))

[1] 0.00 0.25 0.50 0.75 1.00

In [198]:
wbcd_n = as.data.frame(lapply(wbcd[2:31], normalize))

In [199]:
summary(wbcd_n$area_mean)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.1174  0.1729  0.2169  0.2711  1.0000 

##### Data prepartion - creating training and test datasets

In [200]:
wbcd_train <- wbcd_n[1:469, ]
wbcd_test <- wbcd_n[470:569, ]

In [201]:
# Exclude the target variable diagnosis
wbcd_train_labels <- wbcd[1:469, 1]
wbcd_test_labels <- wbcd[470:569, 1]

In [202]:
#Install the class package
install.packages("class")

Installing package into 'C:/Users/Pappu/Documents/R/win-library/3.6'
(as 'lib' is unspecified)



  There is a binary version available but the source version is later:
      binary source needs_compilation
class 7.3-19 7.3-20              TRUE

  Binaries will be installed


Warning message:
"package 'class' is in use and will not be installed"

In [204]:
library(class)

In [205]:
wbcd_test_pred <- knn(train = wbcd_train, test = wbcd_test, cl = wbcd_train_labels, k=21)

##### Evaluating model performance

In [ ]:
#Install the gmodels package
install.packages("gmodels")

In [210]:
library(gmodels)

In [264]:
CrossTable(x=wbcd_test_labels, y=wbcd_test_pred, prop.chisq = FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
                 | wbcd_test_pred 
wbcd_test_labels |    Benign | Malignant | Row Total | 
-----------------|-----------|-----------|-----------|
          Benign |        61 |         0 |        61 | 
                 |     1.000 |     0.000 |     0.610 | 
                 |     0.968 |     0.000 |           | 
                 |     0.610 |     0.000 |           | 
-----------------|-----------|-----------|-----------|
       Malignant |         2 |        37 |        39 | 
                 |     0.051 |     0.949 |     0.390 | 
                 |     0.032 |     1.000 |           | 
                 |     0.020 |     0.370 |           | 
-----------------|-----------|-----------|-----------|
    Column Total |        63 |        37 |       100 | 
           

In [263]:
accuracy = (61+37)/(61+2+37)
accuracy

[1] 0.98

#### Improving model performance
###### -> First employ an alternative method for rescaling numeric data
###### -> Secondly try several different value for the K

In [244]:
# Data Tranformation using z-score standardization
wbcd_z = as.data.frame(scale(wbcd[2:31]))

In [245]:
summary(wbcd_z$area_mean)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.4532 -0.6666 -0.2949  0.0000  0.3632  5.2459 

In [253]:
wbcd_train <- wbcd_z[1:469, ]
wbcd_test <- wbcd_z[470:569, ]

wbcd_train_labels <- wbcd[1:469, 1]
wbcd_test_labels <- wbcd[470:569, 1]

wbcd_test_preds = knn(train =wbcd_train , test = wbcd_test, cl = wbcd_train_labels, k = 21)

In [254]:
CrossTable(x = wbcd_test_labels, y = wbcd_test_preds, prop.chisq = FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
                 | wbcd_test_preds 
wbcd_test_labels |    Benign | Malignant | Row Total | 
-----------------|-----------|-----------|-----------|
          Benign |        61 |         0 |        61 | 
                 |     1.000 |     0.000 |     0.610 | 
                 |     0.924 |     0.000 |           | 
                 |     0.610 |     0.000 |           | 
-----------------|-----------|-----------|-----------|
       Malignant |         5 |        34 |        39 | 
                 |     0.128 |     0.872 |     0.390 | 
                 |     0.076 |     1.000 |           | 
                 |     0.050 |     0.340 |           | 
-----------------|-----------|-----------|-----------|
    Column Total |        66 |        34 |       100 | 
          

##### Here the accurcy is low then the previous

In [262]:
accuracy = (61+34)/(61+5+34)
accuracy

[1] 0.95